In [8]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, TensorDataset
import joblib


In [9]:
# Đọc dữ liệu
df = pd.read_csv("/content/symptom_disease_dataset.csv")  # Đảm bảo file này có sẵn

# Chuyển 'yes'/'no' thành 1/0
for col in df.columns[:-1]:  # bỏ cột "disease"
    df[col] = df[col].map({'yes': 1, 'no': 0})

# Mã hóa nhãn bệnh
le = LabelEncoder()
df["disease"] = le.fit_transform(df["disease"])

# Tách features và labels
X = df.drop(columns="disease").values
y = df["disease"].values
num_classes = len(le.classes_)

# Lưu encoder
joblib.dump(le, "label_encoder.pkl")



['label_encoder.pkl']

In [10]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Chuyển sang tensor
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

# DataLoader
train_ds = TensorDataset(X_train, y_train)
train_dl = DataLoader(train_ds, batch_size=8, shuffle=True)


In [11]:

# Mô hình mạng nơ-ron
class DiseaseModel(nn.Module):
    def __init__(self, input_size, num_classes):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_size, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, num_classes)
        )

    def forward(self, x):
        return self.fc(x)



In [12]:
model = DiseaseModel(input_size=X.shape[1], num_classes=num_classes)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)


In [13]:
# Huấn luyện mô hình
for epoch in range(100):  # 100 epoch
    model.train()
    total_loss = 0
    for xb, yb in train_dl:
        preds = model(xb)
        loss = loss_fn(preds, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")


Epoch 1, Loss: 58.8864
Epoch 2, Loss: 24.6406
Epoch 3, Loss: 16.0899
Epoch 4, Loss: 13.4036
Epoch 5, Loss: 11.1079
Epoch 6, Loss: 9.4931
Epoch 7, Loss: 10.4640
Epoch 8, Loss: 10.2223
Epoch 9, Loss: 9.3137
Epoch 10, Loss: 9.0822
Epoch 11, Loss: 10.1980
Epoch 12, Loss: 9.1364
Epoch 13, Loss: 8.7909
Epoch 14, Loss: 9.2048
Epoch 15, Loss: 8.1493
Epoch 16, Loss: 8.1202
Epoch 17, Loss: 8.5567
Epoch 18, Loss: 7.6673
Epoch 19, Loss: 7.8581
Epoch 20, Loss: 7.5721
Epoch 21, Loss: 8.7421
Epoch 22, Loss: 7.4987
Epoch 23, Loss: 7.8520
Epoch 24, Loss: 7.4754
Epoch 25, Loss: 7.4228
Epoch 26, Loss: 8.9959
Epoch 27, Loss: 9.0438
Epoch 28, Loss: 7.2866
Epoch 29, Loss: 7.5274
Epoch 30, Loss: 7.4011
Epoch 31, Loss: 7.2446
Epoch 32, Loss: 7.1747
Epoch 33, Loss: 7.4417
Epoch 34, Loss: 7.5034
Epoch 35, Loss: 7.4250
Epoch 36, Loss: 7.0681
Epoch 37, Loss: 6.8708
Epoch 38, Loss: 7.0020
Epoch 39, Loss: 7.2464
Epoch 40, Loss: 7.3309
Epoch 41, Loss: 7.0033
Epoch 42, Loss: 6.7557
Epoch 43, Loss: 6.8887
Epoch 44, Lo

In [ ]:
# Lưu mô hình
torch.save(model.state_dict(), "disease_model.pt")
print(" Model and label encoder saved.")


✅ Model and label encoder saved.


In [ ]:
# Đánh giá mô hình trên tập test
model.eval()
with torch.no_grad():
    preds = model(X_test)
    predicted_classes = torch.argmax(preds, dim=1)
    correct = (predicted_classes == y_test).sum().item()
    total = y_test.size(0)
    accuracy = correct / total * 100
    print(f"Test Accuracy: {accuracy:.2f}%")

# In 5 mẫu test đầu tiên: input, label thật và label dự đoán
print("\n Một số dự đoán mẫu kèm dữ liệu đầu vào:")
symptom_names = df.columns[:-1]  # bỏ cột 'disease'
for i in range(5):
    input_symptoms = X_test[i].numpy().astype(int)
    symptoms = [symptom_names[j] for j, val in enumerate(input_symptoms) if val == 1]
    actual_label = le.inverse_transform([y_test[i].item()])[0]
    predicted_label = le.inverse_transform([predicted_classes[i].item()])[0]

    print(f"\n Mẫu {i+1}")
    print(f"  Triệu chứng đầu vào: {', '.join(symptoms) if symptoms else 'Không có'}")
    print(f"  Thực tế: {actual_label}")
    print(f"  Dự đoán: {predicted_label}")


🎯 Test Accuracy: 94.00%

📌 Một số dự đoán mẫu kèm dữ liệu đầu vào:

🧪 Mẫu 1
  Triệu chứng đầu vào: headache, runny_nose, fever, cough, sore_throat, fatigue
  Thực tế: flu
  Dự đoán: flu

🧪 Mẫu 2
  Triệu chứng đầu vào: headache, sore_throat, sneeze, nausea
  Thực tế: stomach_ache
  Dự đoán: stomach_ache

🧪 Mẫu 3
  Triệu chứng đầu vào: headache, runny_nose, fever, cough, sore_throat, fatigue
  Thực tế: covid-19
  Dự đoán: flu

🧪 Mẫu 4
  Triệu chứng đầu vào: headache, fever, cough, sore_throat, sneeze, fatigue
  Thực tế: covid-19
  Dự đoán: covid-19

🧪 Mẫu 5
  Triệu chứng đầu vào: runny_nose, cough, sneeze, nausea
  Thực tế: cold
  Dự đoán: cold
